## Import Packages

In [ ]:
from typing import List, Dict
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import tqdm

import cv2
import albumentations as A
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset, TensorDataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.metrics import FBeta
from pytorch_lightning.loggers import CSVLogger

import torch
import torchvision.models as models
from torch import nn
from torch.optim import AdamW, Adam
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

## Project Directories

In [ ]:
ROOT_DIR = '../input/plant-pathology-2021-fgvc8/'
TRAIN_CSV = 'train.csv'
TRAIN_IMAGES_FOLDER = 'train_images'
TEST_IMAGES_FOLDER = 'test_images'
SAMPLE_SUBMISSION_CSV = 'sample_submission.csv'

## Configurations

In [ ]:
RANDOM_SEED = 42
# Set seed for everythin(numpy, torch and python)

from pytorch_lightning import seed_everything
seed_everything(RANDOM_SEED)

In [ ]:
configurations = {
    "BATCH_SIZE": 128,
    "NUM_WORKERS": 4,
    "IMAGE_HEIGHT": 334, 
    "IMAGE_WIDTH": 334,
    "LEARNING_RATE": 0.003,
    "MAX_EPOCHS": 6,
}

## Data Preparation

In [ ]:
dataset_df = pd.read_csv(os.path.join(ROOT_DIR, TRAIN_CSV))
dataset_df.head()

In [ ]:
dataset_df.info()

In [ ]:
dataset_df.labels.value_counts()

Let's visualize the label count distribution...

In [ ]:
plt.figure(figsize=(16,10))
base_color = sns.color_palette()[0]

ax = sns.countplot(x='labels', data=dataset_df, color=base_color);
ax.set_xticklabels(ax.get_xticklabels(), rotation=60);

In [ ]:
def get_single_labels(unique_labels) -> List[str]:
    """Splitting multi-labels and returning a list of classes"""
    single_labels = []
    for label in unique_labels:
        single_labels += label.split()
        
    single_labels = set(single_labels)
    
    return list(single_labels)

In [ ]:
def get_one_hot_encoded_dataframe(dataset_df):
    # copy dataframe
    dataset_df_copy = dataset_df.copy()
    
    unique_labels = dataset_df_copy.labels.unique()
    
    new_column_names = get_single_labels(unique_labels)
    # initialize columns with zero
    dataset_df_copy[new_column_names] = 0        
    
    # one-hot-encoding using the column names
    for label in unique_labels:                
        label_indices = dataset_df_copy[dataset_df_copy['labels'] == label].index
        splited_labels = label.split()
        dataset_df_copy.loc[label_indices, splited_labels] = 1
    
    return dataset_df_copy

In [ ]:
dataset_df_copy = get_one_hot_encoded_dataframe(dataset_df)
dataset_df_copy.head()

## Visualize the Images with Different Diseases

In [ ]:
def show_images(dataset_df: pd.DataFrame, label_column: str, sample: int=4) -> None:
    fig, axs = plt.subplots(1, sample, figsize=(18, 12))

    df_sample = dataset_df[dataset_df[label_column] == 1].sample(n=sample, random_state=RANDOM_SEED)

    for idx, ax in enumerate(axs):
        image_path = os.path.join(ROOT_DIR, TRAIN_IMAGES_FOLDER, df_sample.iloc[idx, 0])        
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        ax.set_title(f"Label: {df_sample.iloc[idx, 1]}")
        ax.axis('off')

    plt.show()

In [ ]:
show_images(dataset_df=dataset_df_copy, label_column='rust')

In [ ]:
show_images(dataset_df=dataset_df_copy, label_column='frog_eye_leaf_spot')

In [ ]:
show_images(dataset_df=dataset_df_copy, label_column='complex')

In [ ]:
show_images(dataset_df=dataset_df_copy, label_column='healthy')

In [ ]:
show_images(dataset_df=dataset_df_copy, label_column='powdery_mildew')

In [ ]:
show_images(dataset_df=dataset_df_copy, label_column='scab')

## Learn About the Image Shapes

> There are around 18k images, going through each of them is time consuming. So, I decided to select 500 images randomly to get an idea about the image shapes. This helps to decide the image height and width for model training.

In [ ]:
image_names = dataset_df_copy.image.values

random_indices = np.random.randint(low=0, high=len(image_names), size=500)

images_heights = []
images_widths = []
for idx in random_indices:
    image_path = os.path.join(ROOT_DIR, TRAIN_IMAGES_FOLDER, image_names[idx])
    image = cv2.imread(image_path)    
    height, width, _ = image.shape
    images_heights.append(height)
    images_widths.append(width)     
        
    
        
max_height = max(images_heights)
max_width = max(images_widths)
print(f"Max Height: {max_height} || Max Width: {max_width}")

min_height = min(images_heights)
min_width = min(images_widths)
print(f"Min Height: {min_height} || Min Width: {min_width}")

avg_height = sum(images_heights)/len(images_heights)
avg_width = sum(images_widths)/len(images_widths)
print(f"Avg Height: {avg_height} || Avg Width: {avg_width}")

## Prepare Image Dataset for Training

In [ ]:
class ImageDataset(Dataset):
    """ Leaf Disease Dataset """
    def __init__(self,
                image_names: List[str],
                labels: List[List[int]],
                image_dir: str, 
                transforms):        
        self.image_names = image_names
        self.image_dir = image_dir
        self.transforms = transforms                
        self.labels = labels


    def __len__(self) -> int:
        return len(self.image_names)

    def __getitem__(self, idx: int):
        image_path = os.path.join(self.image_dir, self.image_names[idx])           
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)                
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    

        target = self.labels[idx]

        transformed_image = self.transforms(image=image)['image']
        sample = {'image_path': image_path, 'image': transformed_image, 'target': target}

        return sample

In [ ]:
class ImageDataModule(pl.LightningDataModule):
    def __init__(self,
                 df: pd.DataFrame,
                 train_transforms,
                 valid_transforms,
                 image_dir: str,
                 fold_num: int,
                 configurations: Dict[str, int]):
        super().__init__()
        self.df = df
        self.train_transforms = train_transforms
        self.valid_transforms = valid_transforms
        self.image_dir = image_dir
        self.fold_num = fold_num
    
    def setup(self, stage=None) -> None:
        folds = StratifiedKFold(n_splits=5, shuffle=True)
        
        train_indexes, valid_indexes = list(folds.split(self.df, self.df['labels']))[self.fold_num]
        
        print(f"Size of Train Dataset: {len(train_indexes)}")
        print(f"Size of Validation Dataset: {len(valid_indexes)}")
        
        train_df = self.df.iloc[train_indexes]
        valid_df = self.df.iloc[valid_indexes]
        
        self.train_dataset = ImageDataset(image_names=train_df.image.values, 
                                        labels=train_df.iloc[:, 2:].values, 
                                        image_dir=self.image_dir, 
                                        transforms=self.train_transforms,
                                        )

        self.valid_dataset = ImageDataset(image_names=valid_df.image.values, 
                                        labels=valid_df.iloc[:, 2:].values, 
                                        image_dir=self.image_dir, 
                                        transforms=self.valid_transforms,
                                        )
        
        
    def train_dataloader(self):        
        train_loader = DataLoader(
            self.train_dataset,
            batch_size=configurations.get("BATCH_SIZE"),
            num_workers=configurations.get("NUM_WORKERS"),
            shuffle=True,
        )
        return train_loader

    def val_dataloader(self):        
        valid_loader = DataLoader(
            self.valid_dataset,
            batch_size=configurations.get("BATCH_SIZE"),
            num_workers=configurations.get("NUM_WORKERS"),
            shuffle=False,
        )
        return valid_loader

    def test_dataloader(self):
        return None

## Image Augmentation with Albumentation

> To create the baseline model, only image resizing and normalizing is considered.

In [ ]:
train_augs = A.Compose([    
    A.Resize(height=configurations.get("IMAGE_HEIGHT"), width=configurations.get("IMAGE_WIDTH"), p=1.0),    
    A.Normalize(),
    ToTensorV2(),
])

valid_augs = A.Compose([
    A.Resize(height=configurations.get("IMAGE_HEIGHT"), width=configurations.get("IMAGE_WIDTH"), p=1.0),
    A.Normalize(),
    ToTensorV2(),
])

## Preparing `LightningModule` 

In [ ]:
class ClassifierModule(pl.LightningModule):
    def __init__(self, learning_rate=0.003, num_classes=6):
        super().__init__()        
        self.metric = FBeta(num_classes=num_classes, beta=0.5, multilabel=True)
        self.learning_rate = learning_rate
        # Try different architectures
        self.model = models.resnet34(pretrained=True)        
        self.model.fc = nn.Linear(in_features=self.model.fc.in_features, out_features=num_classes)        
        
        
    def forward(self, x):
        batch_size, _, _, _ = x.shape
        x = self.model(x)                
        x = torch.sigmoid(x)
        
        return x.reshape(batch_size, -1)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.001)        

        return optimizer            
    
    def _get_loss(self, y_hat, y): 
        loss = nn.BCELoss()        
        return loss(y_hat.to(torch.float32), y.to(torch.float32))
    
    def training_step(self, batch, batch_idx):
        image = batch['image']
        y = batch['target']
        y_hat = self(image)                           
        
        loss = self._get_loss(y_hat, y)        
        f1_beta_score = self.metric(y_hat, y)
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)        
        self.log('f1_train', f1_beta_score, on_step=True, on_epoch=True, prog_bar=True, logger=True)
                
        return {
            'loss': loss,                        
            'logits': y_hat,
            'target': y,            
        }                 
        
    def validation_step(self, batch, batch_idx):
        image = batch['image']
        y = batch['target']
        y_hat = self(image)
        
        loss = self._get_loss(y_hat, y)
        f1_beta_score = self.metric(y_hat, y)        
        
        self.log('valid_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('f1_valid', f1_beta_score, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return {
            'loss': loss,                        
            'logits': y_hat,
            'target': y,            
        }                            

## Start Training

In [ ]:
data_module = ImageDataModule(df=dataset_df_copy,
                               train_transforms=train_augs,
                               valid_transforms=valid_augs,
                               image_dir= os.path.join(ROOT_DIR, TRAIN_IMAGES_FOLDER),
                               fold_num=0,
                               configurations=configurations)


trainer = pl.Trainer(
        deterministic=True,
        checkpoint_callback=ModelCheckpoint(monitor='train_loss_epoch', save_top_k=1, filename='resnet18-foldnum-0_{epoch}_{valid_loss_epoch:.4f}_{f1_valid_epoch:.4f}', mode='min'),
        gpus=1 if torch.cuda.is_available() else 0,                
        max_epochs=configurations.get("MAX_EPOCHS", 1),
        num_sanity_val_steps=1,        
        weights_summary='top',        
)

lightning = ClassifierModule()

trainer.fit(lightning, data_module)